In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#https://www.kaggle.com/competitions/forecasting-mini-course-sales/overview
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler,
    LabelEncoder, OneHotEncoder, OrdinalEncoder,
    PowerTransformer, QuantileTransformer
)
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate,
    StratifiedKFold, KFold, GridSearchCV, RandomizedSearchCV,
    validation_curve, learning_curve
)


from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier,
    GradientBoostingClassifier, AdaBoostClassifier,
    VotingClassifier, BaggingClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis


from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet,
    SGDRegressor, BayesianRidge, HuberRegressor
)
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor,
    GradientBoostingRegressor, AdaBoostRegressor,
    VotingRegressor, BaggingRegressor
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor


from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture


from sklearn.decomposition import PCA, TruncatedSVD, FastICA
from sklearn.manifold import TSNE
from sklearn.feature_selection import (
    SelectKBest, SelectFromModel, RFE, RFECV,
    chi2, f_classif, f_regression, mutual_info_classif
)


from sklearn.metrics import (
    # Clasificación
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, precision_recall_curve, auc, log_loss,
    
    # Regresión
    mean_squared_error, mean_absolute_error, r2_score,
    mean_squared_log_error, mean_absolute_percentage_error,
    
    # Clustering
    silhouette_score, adjusted_rand_score, calinski_harabasz_score
)

try:
    import xgboost as xgb
    print("✅ XGBoost disponible")
except ImportError:
    print("❌ XGBoost no instalado")

try:
    import lightgbm as lgb
    print("✅ LightGBM disponible")
except ImportError:
    print("❌ LightGBM no instalado")

try:
    from catboost import CatBoostClassifier, CatBoostRegressor
    print("✅ CatBoost disponible")
except ImportError:
    print("❌ CatBoost no instalado")


try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
    print("✅ TensorFlow disponible")
except ImportError:
    print("❌ TensorFlow no instalado")

try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    print("✅ PyTorch disponible")
except ImportError:
    print("❌ PyTorch no instalado")


try:
    import spacy
    print("✅ SpaCy disponible")
except ImportError:
    print("❌ SpaCy no instalado")


try:
    import scipy.stats as stats
    from scipy import stats
    print("✅ SciPy disponible")
except ImportError:
    print("❌ SciPy no instalado")

try:
    import statsmodels.api as sm
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    print("✅ Statsmodels disponible")
except ImportError:
    print("❌ Statsmodels no instalado")


try:
    from prophet import Prophet
    print("✅ Prophet disponible")
except ImportError:
    print("❌ Prophet no instalado")


try:
    import optuna
    print("✅ Optuna disponible")
except ImportError:
    print("❌ Optuna no instalado")


import os
import sys
import json
import pickle
import joblib
from datetime import datetime, timedelta
import itertools
from collections import Counter
import gc


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.width', None)


plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12


sns.set_palette("husl")
sns.set_style("whitegrid")


np.random.seed(1722)


def info_basica(df):
    """Función para explorar un dataset rápidamente"""
    print("=" * 50)
    print("INFORMACIÓN BÁSICA DEL DATASET")
    print("=" * 50)
    print(f"Forma del dataset: {df.shape}")
    print(f"Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print("\nTipos de datos:")
    print(df.dtypes.value_counts())
    print("\nValores nulos:")
    print(df.isnull().sum().sort_values(ascending=False))
    print("\nValores duplicados:", df.duplicated().sum())
    print("\nPrimeras 5 filas:")
    print(df.head())

def plot_distribucion(df, columna, tipo='auto'):
    """Plotea la distribución de una columna"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    if tipo == 'auto':
        tipo = 'continua' if df[columna].dtype in ['float64', 'int64'] else 'categorica'
    
    if tipo == 'continua':
        # Histograma
        ax1.hist(df[columna].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        ax1.set_title(f'Distribución de {columna}')
        ax1.set_xlabel(columna)
        ax1.set_ylabel('Frecuencia')
        
        # Boxplot
        ax2.boxplot(df[columna].dropna())
        ax2.set_title(f'Boxplot de {columna}')
        ax2.set_ylabel(columna)
    else:
        # Gráfico de barras
        df[columna].value_counts().plot(kind='bar', ax=ax1, color='lightcoral')
        ax1.set_title(f'Distribución de {columna}')
        ax1.set_xlabel(columna)
        ax1.set_ylabel('Frecuencia')
        ax1.tick_params(axis='x', rotation=45)
        
        # Gráfico de pie
        df[columna].value_counts().plot(kind='pie', ax=ax2, autopct='%1.1f%%')
        ax2.set_title(f'Proporción de {columna}')
        ax2.set_ylabel('')
    
    plt.tight_layout()
    plt.show()


In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s3e19/train.csv') 
test  = pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv")
test_orig = pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv")
train.head()

In [ ]:
# agrego 1 mes de datos a test que despues voy a usar

december_2021 = train[(train['date'] >= '2021-12-01') & (train['date'] <= '2021-12-31')]
test = pd.concat([december_2021, test], ignore_index=True)

In [ ]:
info_basica(train)

In [ ]:
info_basica(test)

In [ ]:
# num_sold lo que tengo que predecir para 27375 filas osea (27375/365) = 75 id cada dia
# 5 paises , 3 tiendas, 5 productos ,(136950/1826) = 75 id para cada dia , (75/5) = 15 de cada pais ,
# cada tienda vende 5 productos en cada pais cada dia (5 * 3 * 5) = 75 id , productos = (136950/27390) = 5
# fecha desde 01-01-2017 a 31-12-2021 5 años * 365 = 1825 , 1826 por un año bisiesto
# predecir todo el año de ventas del 2022

#por como esta hecha toda sintetica la info lo primero que se me viene en agrupar informacion y enfocar el modelo
#en predecir distribuciones sea nivel pais o dia o tienda o producto , ver si sigue alguna distrib de conteo conocida
# y comparar si año a año cambia

train.describe(include="all")

In [ ]:
test.describe(include="all")

In [ ]:
train["date"].value_counts()

In [ ]:
test["date"].value_counts()

In [ ]:
train1 = train.copy()
test1  = test.copy()

# date lo separo 

train1['date'] = pd.to_datetime(train1['date'])
test1['date'] = pd.to_datetime(test1['date'])


train1['Year'] = train1['date'].dt.year
train1['Quarter'] = train1['date'].dt.quarter
train1['Month'] = train1['date'].dt.month
train1['Day'] = train1['date'].dt.day
train1['DayOfWeek'] = train1['date'].dt.dayofweek  # 0=Lunes, 6=Domingo

test1['Year'] = test1['date'].dt.year
test1['Quarter'] = test1['date'].dt.quarter
test1['Month'] = test1['date'].dt.month
test1['Day'] = test1['date'].dt.day
test1['DayOfWeek'] = test1['date'].dt.dayofweek  # 0=Lunes, 6=Domingo

train1.head(76)

In [ ]:
# voy a prodecer a ver distintos tipos de agrupaciones y estadisticas , para ver de que forma me conviene
# hacer el modelo , si predecir de arriba para abajo o  al reves


# Distribuciones a diferentes niveles de agregación
daily_sales = train1.groupby('date')['num_sold'].sum()
country_daily = train1.groupby(['date', 'country'])['num_sold'].sum()
store_daily = train1.groupby(['date', 'country', 'store'])['num_sold'].sum()
product_daily = train1.groupby(['date', 'product'])['num_sold'].sum()

daily_sales


In [ ]:

# Crear subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Distribuciones de Ventas por Nivel de Agregación', fontsize=16)

# 1. Ventas totales diarias
axes[0,0].hist(daily_sales, bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[0,0].set_title(f'Ventas Totales Diarias\nMedia: {daily_sales.mean():.1f}, Std: {daily_sales.std():.1f}')
axes[0,0].set_xlabel('Ventas Totales por Día')
axes[0,0].set_ylabel('Frecuencia')

# 2. Ventas por país-día
axes[0,1].hist(country_daily, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[0,1].set_title(f'Ventas por País-Día\nMedia: {country_daily.mean():.1f}, Std: {country_daily.std():.1f}')
axes[0,1].set_xlabel('Ventas por País-Día')
axes[0,1].set_ylabel('Frecuencia')

# 3. Ventas por tienda-día
axes[1,0].hist(store_daily, bins=30, alpha=0.7, color='orange', edgecolor='black')
axes[1,0].set_title(f'Ventas por Tienda-Día\nMedia: {store_daily.mean():.1f}, Std: {store_daily.std():.1f}')
axes[1,0].set_xlabel('Ventas por Tienda-Día')
axes[1,0].set_ylabel('Frecuencia')

# 4. Ventas por producto-día
axes[1,1].hist(product_daily, bins=30, alpha=0.7, color='red', edgecolor='black')
axes[1,1].set_title(f'Ventas por Producto-Día\nMedia: {product_daily.mean():.1f}, Std: {product_daily.std():.1f}')
axes[1,1].set_xlabel('Ventas por Producto-Día')
axes[1,1].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

# Estadísticas descriptivas
print("=== ESTADÍSTICAS DESCRIPTIVAS ===")
print(f"\n1. VENTAS TOTALES DIARIAS (75 registros/día):")
print(f"   Min: {daily_sales.min()}, Max: {daily_sales.max()}")
print(f"   Media: {daily_sales.mean():.2f}, Mediana: {daily_sales.median():.2f}")
print(f"   Coef. Variación: {daily_sales.std()/daily_sales.mean():.3f}")

print(f"\n2. VENTAS POR PAÍS-DÍA (15 registros/país/día):")
print(f"   Min: {country_daily.min()}, Max: {country_daily.max()}")
print(f"   Media: {country_daily.mean():.2f}, Mediana: {country_daily.median():.2f}")
print(f"   Coef. Variación: {country_daily.std()/country_daily.mean():.3f}")

print(f"\n3. VENTAS POR TIENDA-DÍA (5 registros/tienda/día):")
print(f"   Min: {store_daily.min()}, Max: {store_daily.max()}")
print(f"   Media: {store_daily.mean():.2f}, Mediana: {store_daily.median():.2f}")
print(f"   Coef. Variación: {store_daily.std()/store_daily.mean():.3f}")

print(f"\n4. VENTAS POR PRODUCTO-DÍA:")
print(f"   Min: {product_daily.min()}, Max: {product_daily.max()}")
print(f"   Media: {product_daily.mean():.2f}, Mediana: {product_daily.median():.2f}")
print(f"   Coef. Variación: {product_daily.std()/product_daily.mean():.3f}")

# Ver la distribución individual (nivel más granular)
print(f"\n5. VENTAS INDIVIDUALES (nivel más granular):")
print(f"   Min: {train1['num_sold'].min()}, Max: {train1['num_sold'].max()}")
print(f"   Media: {train1['num_sold'].mean():.2f}, Mediana: {train1['num_sold'].median():.2f}")
print(f"   Coef. Variación: {train1['num_sold'].std()/train1['num_sold'].mean():.3f}")

# Histograma adicional para el nivel individual
plt.figure(figsize=(10, 6))
plt.hist(train1['num_sold'], bins=50, alpha=0.7, color='purple', edgecolor='black')
plt.title(f'Distribución de Ventas Individuales\nMedia: {train1["num_sold"].mean():.1f}, Std: {train1["num_sold"].std():.1f}')
plt.xlabel('Número de Ventas')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Crear las agregaciones por año
years = [2017, 2018, 2019, 2020, 2021]
colors = ['blue', 'green', 'orange', 'red', 'purple']

# 1. VENTAS POR TIENDA-DÍA POR AÑO
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Distribuciones de Ventas por Tienda-Día por Año', fontsize=16)

for i, year in enumerate(years):
    year_data = train1[train1['Year'] == year]
    store_daily_year = year_data.groupby(['date', 'country', 'store'])['num_sold'].sum()
    
    row = i // 3
    col = i % 3
    
    axes[row, col].hist(store_daily_year, bins=25, alpha=0.7, color=colors[i], edgecolor='black')
    axes[row, col].set_title(f'Año {year}\nMedia: {store_daily_year.mean():.1f}, Std: {store_daily_year.std():.1f}')
    axes[row, col].set_xlabel('Ventas por Tienda-Día')
    axes[row, col].set_ylabel('Frecuencia')
    
    # Agregar línea vertical en la media
    axes[row, col].axvline(store_daily_year.mean(), color='black', linestyle='--', alpha=0.8)

# Remover el subplot extra
axes[1, 2].remove()

plt.tight_layout()
plt.show()

# 2. VENTAS TOTALES DIARIAS POR AÑO
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Distribuciones de Ventas Totales Diarias por Año', fontsize=16)

for i, year in enumerate(years):
    year_data = train1[train1['Year'] == year]
    daily_sales_year = year_data.groupby('date')['num_sold'].sum()
    
    row = i // 3
    col = i % 3
    
    axes[row, col].hist(daily_sales_year, bins=25, alpha=0.7, color=colors[i], edgecolor='black')
    axes[row, col].set_title(f'Año {year}\nMedia: {daily_sales_year.mean():.1f}, Std: {daily_sales_year.std():.1f}')
    axes[row, col].set_xlabel('Ventas Totales Diarias')
    axes[row, col].set_ylabel('Frecuencia')
    
    # Agregar línea vertical en la media
    axes[row, col].axvline(daily_sales_year.mean(), color='black', linestyle='--', alpha=0.8)

# Remover el subplot extra
axes[1, 2].remove()

plt.tight_layout()
plt.show()

# 3. COMPARACIÓN DE ESTADÍSTICAS POR AÑO
print("=== EVOLUCIÓN DE ESTADÍSTICAS POR AÑO ===")
print("\n1. VENTAS POR TIENDA-DÍA:")
print("Año\tMedia\tStd\tCoef.Var\tMin\tMax")
print("-" * 50)

store_stats = []
for year in years:
    year_data = train1[train1['Year'] == year]
    store_daily_year = year_data.groupby(['date', 'country', 'store'])['num_sold'].sum()
    
    mean_val = store_daily_year.mean()
    std_val = store_daily_year.std()
    cv = std_val / mean_val
    min_val = store_daily_year.min()
    max_val = store_daily_year.max()
    
    store_stats.append([year, mean_val, std_val, cv, min_val, max_val])
    print(f"{year}\t{mean_val:.1f}\t{std_val:.1f}\t{cv:.3f}\t\t{min_val}\t{max_val}")

print("\n2. VENTAS TOTALES DIARIAS:")
print("Año\tMedia\tStd\tCoef.Var\tMin\tMax")
print("-" * 50)

daily_stats = []
for year in years:
    year_data = train1[train1['Year'] == year]
    daily_sales_year = year_data.groupby('date')['num_sold'].sum()
    
    mean_val = daily_sales_year.mean()
    std_val = daily_sales_year.std()
    cv = std_val / mean_val
    min_val = daily_sales_year.min()
    max_val = daily_sales_year.max()
    
    daily_stats.append([year, mean_val, std_val, cv, min_val, max_val])
    print(f"{year}\t{mean_val:.1f}\t{std_val:.1f}\t{cv:.3f}\t\t{min_val}\t{max_val}")

# 4. GRÁFICOS DE EVOLUCIÓN TEMPORAL
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Evolución de medias
store_means = [stats[1] for stats in store_stats]
daily_means = [stats[1] for stats in daily_stats]

axes[0].plot(years, store_means, 'o-', color='blue', linewidth=2, markersize=8, label='Tienda-Día')
axes[0].plot(years, daily_means, 's-', color='red', linewidth=2, markersize=8, label='Total Diario')
axes[0].set_title('Evolución de Medias por Año')
axes[0].set_xlabel('Año')
axes[0].set_ylabel('Media de Ventas')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Evolución de coeficientes de variación
store_cvs = [stats[3] for stats in store_stats]
daily_cvs = [stats[3] for stats in daily_stats]

axes[1].plot(years, store_cvs, 'o-', color='blue', linewidth=2, markersize=8, label='Tienda-Día')
axes[1].plot(years, daily_cvs, 's-', color='red', linewidth=2, markersize=8, label='Total Diario')
axes[1].set_title('Evolución de Coeficiente de Variación por Año')
axes[1].set_xlabel('Año')
axes[1].set_ylabel('Coeficiente de Variación')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 5. TEST DE NORMALIDAD BÁSICO (opcional)
from scipy import stats as scipy_stats

print("\n=== TESTS DE FORMA DE DISTRIBUCIÓN ===")
print("\nPara detectar si las distribuciones cambian de forma:")
print("- Si Coef.Var ≈ 1: podría ser Poisson")
print("- Si Coef.Var > 1: podría ser Binomial Negativa")
print("- Si Coef.Var << 1: podría tender a Normal")

for year in years:
    year_data = train1[train1['Year'] == year]
    store_daily_year = year_data.groupby(['date', 'country', 'store'])['num_sold'].sum()
    
    cv = store_daily_year.std() / store_daily_year.mean()
    
    if abs(cv - 1) < 0.1:
        distribution_type = "Poisson-like"
    elif cv > 1.2:
        distribution_type = "Overdispersed (Binomial Negativa?)"
    elif cv < 0.5:
        distribution_type = "Underdispersed (Normal?)"
    else:
        distribution_type = "Intermedio"
    
    print(f"Año {year} - Tienda-Día: CV={cv:.3f} -> {distribution_type}")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Asegurarse de que la columna date sea datetime si no lo está
train1['date'] = pd.to_datetime(train1['date'])

# Para los días de la semana, crear nombres legibles
day_names = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 
             4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
train1['day_name'] = train1['DayOfWeek'].map(day_names)

years = [2017, 2018, 2019, 2020, 2021]
colors = ['blue', 'green', 'orange', 'red', 'purple']

# ===== 1. ANÁLISIS POR PAÍS AÑO A AÑO =====
print("=== ANÁLISIS POR PAÍS AÑO A AÑO ===")

# Obtener los países únicos
countries = train1['country'].unique()
print(f"Países en el dataset: {countries}")

# Crear figura para distribuciones por país
fig, axes = plt.subplots(len(countries), len(years), figsize=(20, 4*len(countries)))
fig.suptitle('Distribuciones de Ventas por País y Año (Tienda-Día)', fontsize=16)

country_stats = {}
for i, country in enumerate(countries):
    country_stats[country] = []
    for j, year in enumerate(years):
        # Filtrar por país y año
        country_year_data = train1[(train1['country'] == country) & (train1['Year'] == year)]
        store_daily = country_year_data.groupby(['date', 'store'])['num_sold'].sum()
        
        # Crear histograma
        axes[i, j].hist(store_daily, bins=20, alpha=0.7, color=colors[j], edgecolor='black')
        axes[i, j].set_title(f'{country} - {year}\nMedia: {store_daily.mean():.1f}')
        axes[i, j].set_xlabel('Ventas Tienda-Día')
        if j == 0:  # Solo en la primera columna
            axes[i, j].set_ylabel(f'{country}\nFrecuencia')
        
        # Guardar estadísticas
        country_stats[country].append({
            'year': year,
            'mean': store_daily.mean(),
            'std': store_daily.std(),
            'cv': store_daily.std() / store_daily.mean(),
            'count': len(store_daily)
        })

plt.tight_layout()
plt.show()

# Tabla de estadísticas por país
print("\nESTADÍSTICAS POR PAÍS:")
for country in countries:
    print(f"\n{country.upper()}:")
    print("Año\tMedia\tStd\tCoef.Var\tN_obs")
    print("-" * 40)
    for stat in country_stats[country]:
        print(f"{stat['year']}\t{stat['mean']:.1f}\t{stat['std']:.1f}\t{stat['cv']:.3f}\t\t{stat['count']}")

# ===== 2. ANÁLISIS POR DÍA DE LA SEMANA AÑO A AÑO =====
print("\n\n=== ANÁLISIS POR DÍA DE LA SEMANA AÑO A AÑO ===")

# Crear figura para distribuciones por día de la semana
fig, axes = plt.subplots(7, len(years), figsize=(20, 28))
fig.suptitle('Distribuciones de Ventas por Día de la Semana y Año (Tienda-Día)', fontsize=16)

day_stats = {}
days_order = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

for i, day in enumerate(days_order):
    day_stats[day] = []
    for j, year in enumerate(years):
        # Filtrar por día de la semana y año
        day_year_data = train1[(train1['day_name'] == day) & (train1['Year'] == year)]
        store_daily = day_year_data.groupby(['date', 'country', 'store'])['num_sold'].sum()
        
        # Crear histograma
        axes[i, j].hist(store_daily, bins=20, alpha=0.7, color=colors[j], edgecolor='black')
        axes[i, j].set_title(f'{day} - {year}\nMedia: {store_daily.mean():.1f}')
        axes[i, j].set_xlabel('Ventas Tienda-Día')
        if j == 0:  # Solo en la primera columna
            axes[i, j].set_ylabel(f'{day}\nFrecuencia')
        
        # Guardar estadísticas
        day_stats[day].append({
            'year': year,
            'mean': store_daily.mean(),
            'std': store_daily.std(),
            'cv': store_daily.std() / store_daily.mean(),
            'count': len(store_daily)
        })

plt.tight_layout()
plt.show()

# Tabla de estadísticas por día de la semana
print("\nESTADÍSTICAS POR DÍA DE LA SEMANA:")
for day in days_order:
    print(f"\n{day.upper()}:")
    print("Año\tMedia\tStd\tCoef.Var\tN_obs")
    print("-" * 40)
    for stat in day_stats[day]:
        print(f"{stat['year']}\t{stat['mean']:.1f}\t{stat['std']:.1f}\t{stat['cv']:.3f}\t\t{stat['count']}")

# ===== 3. GRÁFICOS DE EVOLUCIÓN TEMPORAL =====

# 3.1 Evolución por país
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

for country in countries:
    means = [stat['mean'] for stat in country_stats[country]]
    cvs = [stat['cv'] for stat in country_stats[country]]
    
    axes[0].plot(years, means, 'o-', linewidth=2, markersize=8, label=country)
    axes[1].plot(years, cvs, 'o-', linewidth=2, markersize=8, label=country)

axes[0].set_title('Evolución de Medias por País')
axes[0].set_xlabel('Año')
axes[0].set_ylabel('Media de Ventas (Tienda-Día)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_title('Evolución de Coef. Variación por País')
axes[1].set_xlabel('Año')
axes[1].set_ylabel('Coeficiente de Variación')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 3.2 Evolución por día de la semana
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

for day in days_order:
    means = [stat['mean'] for stat in day_stats[day]]
    cvs = [stat['cv'] for stat in day_stats[day]]
    
    axes[0].plot(years, means, 'o-', linewidth=2, markersize=6, label=day)
    axes[1].plot(years, cvs, 'o-', linewidth=2, markersize=6, label=day)

axes[0].set_title('Evolución de Medias por Día de la Semana')
axes[0].set_xlabel('Año')
axes[0].set_ylabel('Media de Ventas (Tienda-Día)')
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].grid(True, alpha=0.3)

axes[1].set_title('Evolución de Coef. Variación por Día de la Semana')
axes[1].set_xlabel('Año')
axes[1].set_ylabel('Coeficiente de Variación')
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ===== 4. ANÁLISIS DE PATRONES =====
print("\n=== ANÁLISIS DE PATRONES ===")

# 4.1 Ranking promedio por país (todos los años)
print("\nRANKING PROMEDIO DE VENTAS POR PAÍS:")
country_overall = []
for country in countries:
    overall_mean = np.mean([stat['mean'] for stat in country_stats[country]])
    country_overall.append((country, overall_mean))

country_overall.sort(key=lambda x: x[1], reverse=True)
for i, (country, mean) in enumerate(country_overall, 1):
    print(f"{i}. {country}: {mean:.1f}")

# 4.2 Ranking promedio por día de la semana
print("\nRANKING PROMEDIO DE VENTAS POR DÍA:")
day_overall = []
for day in days_order:
    overall_mean = np.mean([stat['mean'] for stat in day_stats[day]])
    day_overall.append((day, overall_mean))

day_overall.sort(key=lambda x: x[1], reverse=True)
for i, (day, mean) in enumerate(day_overall, 1):
    print(f"{i}. {day}: {mean:.1f}")

In [ ]:
years = [2017, 2018, 2019, 2020, 2021]
colors = ['blue', 'green', 'orange', 'red', 'purple']

# Obtener valores únicos
countries = train1['country'].unique()
products = train1['product'].unique()
stores = train1['store'].unique()

print(f"Países: {len(countries)} - {countries}")
print(f"Productos: {len(products)} - {products}")
print(f"Tiendas: {len(stores)} - {stores}")

# ===== 1. ANÁLISIS POR PRODUCTO-PAÍS-AÑO =====
print("\n=== ANÁLISIS POR PRODUCTO-PAÍS-AÑO ===")

# Para cada país, mostrar productos por año
for country in countries:
    print(f"\n{'='*50}")
    print(f"ANÁLISIS PARA {country.upper()}")
    print(f"{'='*50}")
    
    fig, axes = plt.subplots(len(products), len(years), figsize=(20, 4*len(products)))
    fig.suptitle(f'Distribuciones de Ventas por Producto y Año - {country}', fontsize=16)
    
    product_stats = {}
    
    for i, product in enumerate(products):
        product_stats[product] = []
        
        for j, year in enumerate(years):
            # Filtrar por país, producto y año
            filtered_data = train1[(train1['country'] == country) & 
                                 (train1['product'] == product) & 
                                 (train1['Year'] == year)]
            
            # Agrupar por fecha y tienda (cada producto-tienda-día)
            daily_sales = filtered_data.groupby(['date', 'store'])['num_sold'].sum()
            
            # Si hay un solo producto, usar axes unidimensional
            if len(products) == 1:
                ax = axes[j]
            else:
                ax = axes[i, j]
            
            # Crear histograma
            ax.hist(daily_sales, bins=15, alpha=0.7, color=colors[j], edgecolor='black')
            ax.set_title(f'{product}\n{year} - Media: {daily_sales.mean():.1f}')
            ax.set_xlabel('Ventas Diarias')
            
            if j == 0:  # Solo en la primera columna
                if len(products) == 1:
                    ax.set_ylabel('Frecuencia')
                else:
                    ax.set_ylabel(f'{product}\nFrecuencia')
            
            # Guardar estadísticas
            product_stats[product].append({
                'year': year,
                'mean': daily_sales.mean(),
                'std': daily_sales.std(),
                'cv': daily_sales.std() / daily_sales.mean() if daily_sales.mean() > 0 else 0,
                'count': len(daily_sales)
            })
    
    plt.tight_layout()
    plt.show()
    
    # Tabla de estadísticas por producto en este país
    print(f"\nESTADÍSTICAS DE PRODUCTOS EN {country}:")
    for product in products:
        print(f"\n{product}:")
        print("Año\tMedia\tStd\tCoef.Var\tN_obs")
        print("-" * 40)
        for stat in product_stats[product]:
            print(f"{stat['year']}\t{stat['mean']:.1f}\t{stat['std']:.1f}\t{stat['cv']:.3f}\t\t{stat['count']}")
    
    # Gráfico de evolución para este país
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    for product in products:
        means = [stat['mean'] for stat in product_stats[product]]
        cvs = [stat['cv'] for stat in product_stats[product]]
        
        axes[0].plot(years, means, 'o-', linewidth=2, markersize=8, label=product)
        axes[1].plot(years, cvs, 'o-', linewidth=2, markersize=8, label=product)
    
    axes[0].set_title(f'Evolución de Medias por Producto - {country}')
    axes[0].set_xlabel('Año')
    axes[0].set_ylabel('Media de Ventas')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].set_title(f'Evolución de Coef. Variación por Producto - {country}')
    axes[1].set_xlabel('Año')
    axes[1].set_ylabel('Coeficiente de Variación')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# ===== 2. ANÁLISIS POR TIENDA-AÑO =====
print(f"\n{'='*60}")
print("ANÁLISIS POR TIENDA-AÑO")
print(f"{'='*60}")

# Crear figura para distribuciones por tienda
fig, axes = plt.subplots(len(stores), len(years), figsize=(20, 4*len(stores)))
fig.suptitle('Distribuciones de Ventas por Tienda y Año', fontsize=16)

store_stats = {}

for i, store in enumerate(stores):
    store_stats[store] = []
    
    for j, year in enumerate(years):
        # Filtrar por tienda y año
        store_year_data = train1[(train1['store'] == store) & (train1['Year'] == year)]
        # Agrupar por fecha, país y producto (cada tienda-día)
        daily_sales = store_year_data.groupby(['date', 'country', 'product'])['num_sold'].sum()
        
        # Si hay una sola tienda, usar axes unidimensional
        if len(stores) == 1:
            ax = axes[j]
        else:
            ax = axes[i, j]
        
        # Crear histograma
        ax.hist(daily_sales, bins=20, alpha=0.7, color=colors[j], edgecolor='black')
        ax.set_title(f'{store} - {year}\nMedia: {daily_sales.mean():.1f}')
        ax.set_xlabel('Ventas Diarias')
        
        if j == 0:  # Solo en la primera columna
            if len(stores) == 1:
                ax.set_ylabel('Frecuencia')
            else:
                ax.set_ylabel(f'{store}\nFrecuencia')
        
        # Guardar estadísticas
        store_stats[store].append({
            'year': year,
            'mean': daily_sales.mean(),
            'std': daily_sales.std(),
            'cv': daily_sales.std() / daily_sales.mean() if daily_sales.mean() > 0 else 0,
            'count': len(daily_sales)
        })

plt.tight_layout()
plt.show()

# Tabla de estadísticas por tienda
print("\nESTADÍSTICAS POR TIENDA:")
for store in stores:
    print(f"\n{store.upper()}:")
    print("Año\tMedia\tStd\tCoef.Var\tN_obs")
    print("-" * 40)
    for stat in store_stats[store]:
        print(f"{stat['year']}\t{stat['mean']:.1f}\t{stat['std']:.1f}\t{stat['cv']:.3f}\t\t{stat['count']}")

# Gráfico de evolución por tienda
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

for store in stores:
    means = [stat['mean'] for stat in store_stats[store]]
    cvs = [stat['cv'] for stat in store_stats[store]]
    
    axes[0].plot(years, means, 'o-', linewidth=2, markersize=8, label=store)
    axes[1].plot(years, cvs, 'o-', linewidth=2, markersize=8, label=store)

axes[0].set_title('Evolución de Medias por Tienda')
axes[0].set_xlabel('Año')
axes[0].set_ylabel('Media de Ventas')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_title('Evolución de Coef. Variación por Tienda')
axes[1].set_xlabel('Año')
axes[1].set_ylabel('Coeficiente de Variación')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ===== 3. RANKINGS Y ANÁLISIS COMPARATIVO =====
print(f"\n{'='*60}")
print("RANKINGS Y ANÁLISIS COMPARATIVO")
print(f"{'='*60}")

# 3.1 Ranking de productos por país (promedio todos los años)
print("\nRANKING DE PRODUCTOS POR PAÍS (Promedio 2017-2021):")
for country in countries:
    print(f"\n{country.upper()}:")
    country_data = train1[train1['country'] == country]
    product_means = country_data.groupby('product')['num_sold'].mean().sort_values(ascending=False)
    
    for i, (product, mean) in enumerate(product_means.items(), 1):
        print(f"  {i}. {product}: {mean:.1f}")

# 3.2 Ranking de tiendas (promedio todos los años)
print(f"\nRANKING DE TIENDAS (Promedio 2017-2021):")
store_means = train1.groupby('store')['num_sold'].mean().sort_values(ascending=False)
for i, (store, mean) in enumerate(store_means.items(), 1):
    print(f"  {i}. {store}: {mean:.1f}")

# 3.3 Productos más consistentes (menor CV)
print(f"\nPRODUCTOS MÁS CONSISTENTES (Menor Coef. Variación):")
product_consistency = train1.groupby('product')['num_sold'].agg(['mean', 'std'])
product_consistency['cv'] = product_consistency['std'] / product_consistency['mean']
product_consistency = product_consistency.sort_values('cv')

for i, (product, stats) in enumerate(product_consistency.iterrows(), 1):
    print(f"  {i}. {product}: CV={stats['cv']:.3f} (Media: {stats['mean']:.1f})")


In [ ]:
# de todas estas estadisticas se obtuvo informacion muy relevante
# lo que mas destaco es ver una distribucion bimodal poisson-normal  o binomial_neg-normal en varios lados, 
# ejemplo la distrib de ventas por producto/año/pais
# en la distrib por tienda y año se ve como todas comparten la misma tendencia cada año
# la distrib por dia y pais se ve las ventas de lu-ju una forma y el fin de semana como cambia el viernes un 5 a 7% aprox mas que el jueves
# el sabado entre 3 a 5 % mas y el domingo 6 a 8 % mas
# cada pais sus propias ventas pero parecen multiplos de una base ,porque respetan proporciones

# lo mismo que pasa abajo , se de tienda en tienda , teniendo a Kaggle Learn como base
#   Arg   Canada(Arg/4.1) Estonia(Arg/2.1)  Japon(Arg/3.6) España(Arg/2.6)
# 1  82.3      83.7          83.2              81.6           82.4
# 2  81.2      82.5          82.2              80.8           81.3
# 3  68.8      69.8          69.5              68.1           68.8
# 4  62.6      63.4          63.2              61.9           62.4
# 5  12.4      12.9          12.8              12.6           12.7



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import minimize

# ===== 1. ANÁLISIS DE SEPARACIÓN CON THRESHOLD =====
def analyze_threshold_split(data, threshold=74):
    """Analiza la separación de datos en dos grupos usando un threshold"""
    
    low_regime = data[data <= threshold]
    high_regime = data[data > threshold]
    
    total_count = len(data)
    low_count = len(low_regime)
    high_count = len(high_regime)
    
    low_prop = low_count / total_count
    high_prop = high_count / total_count
    
    results = {
        'threshold': threshold,
        'total_count': total_count,
        'low_regime': {
            'count': low_count,
            'proportion': low_prop,
            'mean': low_regime.mean() if len(low_regime) > 0 else 0,
            'std': low_regime.std() if len(low_regime) > 0 else 0,
            'data': low_regime
        },
        'high_regime': {
            'count': high_count,
            'proportion': high_prop,
            'mean': high_regime.mean() if len(high_regime) > 0 else 0,
            'std': high_regime.std() if len(high_regime) > 0 else 0,
            'data': high_regime
        }
    }
    
    return results

# ===== 2. TEST DE DISTRIBUCIONES POR RÉGIMEN =====
def test_distributions(data, regime_name):
    """Testa qué distribución se ajusta mejor a los datos"""
    
    if len(data) < 10:
        return f"Datos insuficientes para {regime_name}"
    
    results = {}
    
    # Test Poisson (solo para enteros positivos)
    if regime_name == "low" and all(data >= 0) and all(data == data.astype(int)):
        try:
            poisson_param = stats.poisson.fit(data, method='MLE')[0]
            poisson_ks = stats.kstest(data, lambda x: stats.poisson.cdf(x, poisson_param))
            results['poisson'] = {
                'param': poisson_param,
                'ks_stat': poisson_ks.statistic,
                'p_value': poisson_ks.pvalue
            }
        except:
            results['poisson'] = {'error': 'No se pudo ajustar Poisson'}
    
    # Test Normal
    try:
        normal_params = stats.norm.fit(data)
        normal_ks = stats.kstest(data, lambda x: stats.norm.cdf(x, *normal_params))
        results['normal'] = {
            'params': normal_params,
            'ks_stat': normal_ks.statistic,
            'p_value': normal_ks.pvalue
        }
    except:
        results['normal'] = {'error': 'No se pudo ajustar Normal'}
    
    # Test Gamma (para datos positivos)
    if all(data > 0):
        try:
            gamma_params = stats.gamma.fit(data)
            gamma_ks = stats.kstest(data, lambda x: stats.gamma.cdf(x, *gamma_params))
            results['gamma'] = {
                'params': gamma_params,
                'ks_stat': gamma_ks.statistic,
                'p_value': gamma_ks.pvalue
            }
        except:
            results['gamma'] = {'error': 'No se pudo ajustar Gamma'}
    
    return results

# ===== 3. ANÁLISIS PARA UN PRODUCTO ESPECÍFICO =====
def analyze_product_mixture(train1, product_name, country_name=None, threshold=74):
    """Analiza mixture distribution para un producto específico"""
    
    print(f"\n{'='*60}")
    print(f"ANÁLISIS DE MIXTURE DISTRIBUTION")
    print(f"Producto: {product_name}")
    if country_name:
        print(f"País: {country_name}")
    print(f"Threshold: {threshold}")
    print(f"{'='*60}")
    
    # Filtrar datos
    if country_name:
        data = train1[(train1['product'] == product_name) & 
                     (train1['country'] == country_name)]['num_sold']
    else:
        data = train1[train1['product'] == product_name]['num_sold']
    
    if len(data) == 0:
        print("No se encontraron datos para los filtros especificados")
        return
    
    # Análisis de threshold
    analysis = analyze_threshold_split(data, threshold)
    
    print(f"\nRESULTADOS DE SEPARACIÓN:")
    print(f"Total observaciones: {analysis['total_count']}")
    print(f"\nRÉGIMEN BAJO (≤{threshold}):")
    print(f"  Observaciones: {analysis['low_regime']['count']} ({analysis['low_regime']['proportion']:.1%})")
    print(f"  Media: {analysis['low_regime']['mean']:.2f}")
    print(f"  Std: {analysis['low_regime']['std']:.2f}")
    
    print(f"\nRÉGIMEN ALTO (>{threshold}):")
    print(f"  Observaciones: {analysis['high_regime']['count']} ({analysis['high_regime']['proportion']:.1%})")
    print(f"  Media: {analysis['high_regime']['mean']:.2f}")
    print(f"  Std: {analysis['high_regime']['std']:.2f}")
    
    # Visualización
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Histograma completo
    axes[0,0].hist(data, bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[0,0].axvline(threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {threshold}')
    axes[0,0].set_title(f'Distribución Completa - {product_name}')
    axes[0,0].set_xlabel('Ventas')
    axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].legend()
    
    # Régimen bajo con Poisson ajustada
    if len(analysis['low_regime']['data']) > 0:
        low_data = analysis['low_regime']['data']
        n, bins, patches = axes[0,1].hist(low_data, bins=20, alpha=0.7, color='green', edgecolor='black', density=True)
        
        # Ajustar Poisson
        poisson_lambda = low_data.mean()
        x_poisson = np.arange(0, int(low_data.max()) + 1)
        y_poisson = stats.poisson.pmf(x_poisson, poisson_lambda)
        axes[0,1].plot(x_poisson, y_poisson, 'ro-', linewidth=2, markersize=4, label=f'Poisson(λ={poisson_lambda:.2f})')
        
        axes[0,1].set_title(f'Régimen Bajo (≤{threshold}) vs Poisson')
        axes[0,1].set_xlabel('Ventas')
        axes[0,1].set_ylabel('Densidad')
        axes[0,1].legend()
    
    # Régimen alto con Normal ajustada
    if len(analysis['high_regime']['data']) > 0:
        high_data = analysis['high_regime']['data']
        n, bins, patches = axes[1,0].hist(high_data, bins=20, alpha=0.7, color='orange', edgecolor='black', density=True)
        
        # Ajustar Normal
        normal_mu = high_data.mean()
        normal_sigma = high_data.std()
        x_normal = np.linspace(high_data.min(), high_data.max(), 100)
        y_normal = stats.norm.pdf(x_normal, normal_mu, normal_sigma)
        axes[1,0].plot(x_normal, y_normal, 'b-', linewidth=2, label=f'Normal(μ={normal_mu:.1f}, σ={normal_sigma:.1f})')
        
        axes[1,0].set_title(f'Régimen Alto (>{threshold}) vs Normal')
        axes[1,0].set_xlabel('Ventas')
        axes[1,0].set_ylabel('Densidad')
        axes[1,0].legend()
    
    # Comparación de distribuciones
    axes[1,1].hist(analysis['low_regime']['data'], bins=20, alpha=0.5, color='green', label=f'Bajo (≤{threshold})', density=True)
    axes[1,1].hist(analysis['high_regime']['data'], bins=20, alpha=0.5, color='orange', label=f'Alto (>{threshold})', density=True)
    axes[1,1].set_title('Comparación de Regímenes (Densidad)')
    axes[1,1].set_xlabel('Ventas')
    axes[1,1].set_ylabel('Densidad')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Test distribuciones
    if len(analysis['low_regime']['data']) > 10:
        print(f"\nTEST DE DISTRIBUCIONES - RÉGIMEN BAJO:")
        low_tests = test_distributions(analysis['low_regime']['data'], "low")
        for dist_name, result in low_tests.items():
            if isinstance(result, dict) and 'p_value' in result:
                print(f"  {dist_name.upper()}: KS p-value = {result['p_value']:.4f}")
                if dist_name == 'poisson':
                    print(f"    Lambda estimado: {result['param']:.2f}")
                elif dist_name == 'normal':
                    print(f"    Media: {result['params'][0]:.2f}, Std: {result['params'][1]:.2f}")
    
    if len(analysis['high_regime']['data']) > 10:
        print(f"\nTEST DE DISTRIBUCIONES - RÉGIMEN ALTO:")
        high_tests = test_distributions(analysis['high_regime']['data'], "high")
        for dist_name, result in high_tests.items():
            if isinstance(result, dict) and 'p_value' in result:
                print(f"  {dist_name.upper()}: KS p-value = {result['p_value']:.4f}")
                if dist_name == 'normal':
                    print(f"    Media: {result['params'][0]:.2f}, Std: {result['params'][1]:.2f}")
    
    return analysis

# ===== 4. ANÁLISIS PARA MÚLTIPLES THRESHOLDS =====
def find_optimal_threshold(data, threshold_range=range(50, 100, 5)):
    """Encuentra el threshold óptimo basado en criterios estadísticos"""
    
    results = []
    
    for threshold in threshold_range:
        analysis = analyze_threshold_split(data, threshold)
        
        # Criterios para evaluar la separación
        # 1. Balance en las proporciones (evitar que un grupo sea muy pequeño)
        balance_score = 1 - abs(0.5 - analysis['low_regime']['proportion'])
        
        # 2. Diferencia en medias (queremos que los grupos sean bien diferentes)
        if analysis['high_regime']['count'] > 0 and analysis['low_regime']['count'] > 0:
            mean_diff = abs(analysis['high_regime']['mean'] - analysis['low_regime']['mean'])
        else:
            mean_diff = 0
        
        # 3. Ratio de medias
        if analysis['low_regime']['mean'] > 0:
            mean_ratio = analysis['high_regime']['mean'] / analysis['low_regime']['mean']
        else:
            mean_ratio = 0
        
        results.append({
            'threshold': threshold,
            'low_prop': analysis['low_regime']['proportion'],
            'high_prop': analysis['high_regime']['proportion'],
            'low_mean': analysis['low_regime']['mean'],
            'high_mean': analysis['high_regime']['mean'],
            'mean_diff': mean_diff,
            'mean_ratio': mean_ratio,
            'balance_score': balance_score
        })
    
    results_df = pd.DataFrame(results)
    return results_df

# ===== EJECUCIÓN DEL ANÁLISIS =====

# Obtener lista de productos
products = train1['product'].unique()
countries = train1['country'].unique()

print("PRODUCTOS DISPONIBLES:")
for i, product in enumerate(products):
    print(f"{i+1}. {product}")

print("\nPAÍSES DISPONIBLES:")
for i, country in enumerate(countries):
    print(f"{i+1}. {country}")

# Análisis para el primer producto como ejemplo
if len(products) > 0:
    example_product = products[0]
    example_country = countries[0]
    
    print(f"\n🔍 ANÁLISIS DE EJEMPLO:")
    print(f"Producto: {example_product}")
    print(f"País: {example_country}")
    
    # Análisis con threshold 74
    analysis_74 = analyze_product_mixture(train1, example_product, example_country, threshold=74)
    
    # Buscar threshold óptimo
    product_data = train1[(train1['product'] == example_product) & 
                         (train1['country'] == example_country)]['num_sold']
    
    print(f"\n🎯 BÚSQUEDA DE THRESHOLD ÓPTIMO:")
    threshold_analysis = find_optimal_threshold(product_data)
    
    # Mostrar los mejores thresholds
    threshold_analysis['score'] = (threshold_analysis['balance_score'] * 
                                  threshold_analysis['mean_diff'] / 100)
    
    best_thresholds = threshold_analysis.nlargest(3, 'score')
    print("\nTOP 3 THRESHOLDS:")
    for idx, row in best_thresholds.iterrows():
        print(f"  Threshold: {row['threshold']}")
        print(f"    Proporción bajo: {row['low_prop']:.1%}, alto: {row['high_prop']:.1%}")
        print(f"    Media bajo: {row['low_mean']:.1f}, alto: {row['high_mean']:.1f}")
        print(f"    Ratio medias: {row['mean_ratio']:.2f}")
        print()



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import minimize

# ===== 1. ANÁLISIS DE SEPARACIÓN CON THRESHOLD =====
def analyze_threshold_split(data, threshold=74):
    """Analiza la separación de datos en dos grupos usando un threshold"""
    
    low_regime = data[data <= threshold]
    high_regime = data[data > threshold]
    
    total_count = len(data)
    low_count = len(low_regime)
    high_count = len(high_regime)
    
    low_prop = low_count / total_count
    high_prop = high_count / total_count
    
    results = {
        'threshold': threshold,
        'total_count': total_count,
        'low_regime': {
            'count': low_count,
            'proportion': low_prop,
            'mean': low_regime.mean() if len(low_regime) > 0 else 0,
            'std': low_regime.std() if len(low_regime) > 0 else 0,
            'data': low_regime
        },
        'high_regime': {
            'count': high_count,
            'proportion': high_prop,
            'mean': high_regime.mean() if len(high_regime) > 0 else 0,
            'std': high_regime.std() if len(high_regime) > 0 else 0,
            'data': high_regime
        }
    }
    
    return results

# ===== 2. TEST DE DISTRIBUCIONES POR RÉGIMEN =====
def test_distributions(data, regime_name):
    """Testa qué distribución se ajusta mejor a los datos"""
    
    if len(data) < 10:
        return f"Datos insuficientes para {regime_name}"
    
    results = {}
    
    # Test Poisson (solo para enteros positivos)
    if regime_name == "low" and all(data >= 0) and all(data == data.astype(int)):
        try:
            poisson_param = stats.poisson.fit(data, method='MLE')[0]
            poisson_ks = stats.kstest(data, lambda x: stats.poisson.cdf(x, poisson_param))
            results['poisson'] = {
                'param': poisson_param,
                'ks_stat': poisson_ks.statistic,
                'p_value': poisson_ks.pvalue
            }
        except:
            results['poisson'] = {'error': 'No se pudo ajustar Poisson'}
    
    # Test Normal
    try:
        normal_params = stats.norm.fit(data)
        normal_ks = stats.kstest(data, lambda x: stats.norm.cdf(x, *normal_params))
        results['normal'] = {
            'params': normal_params,
            'ks_stat': normal_ks.statistic,
            'p_value': normal_ks.pvalue
        }
    except:
        results['normal'] = {'error': 'No se pudo ajustar Normal'}
    
    # Test Gamma (para datos positivos)
    if all(data > 0):
        try:
            gamma_params = stats.gamma.fit(data)
            gamma_ks = stats.kstest(data, lambda x: stats.gamma.cdf(x, *gamma_params))
            results['gamma'] = {
                'params': gamma_params,
                'ks_stat': gamma_ks.statistic,
                'p_value': gamma_ks.pvalue
            }
        except:
            results['gamma'] = {'error': 'No se pudo ajustar Gamma'}
    
    return results

# ===== 3. ANÁLISIS PARA UN PRODUCTO ESPECÍFICO =====
def analyze_product_mixture(train1, product_name, country_name=None, threshold=74):
    """Analiza mixture distribution para un producto específico"""
    
    print(f"\n{'='*60}")
    print(f"ANÁLISIS DE MIXTURE DISTRIBUTION")
    print(f"Producto: {product_name}")
    if country_name:
        print(f"País: {country_name}")
    print(f"Threshold: {threshold}")
    print(f"{'='*60}")
    
    # Filtrar datos
    if country_name:
        data = train1[(train1['product'] == product_name) & 
                     (train1['country'] == country_name)]['num_sold']
    else:
        data = train1[train1['product'] == product_name]['num_sold']
    
    if len(data) == 0:
        print("No se encontraron datos para los filtros especificados")
        return
    
    # Análisis de threshold
    analysis = analyze_threshold_split(data, threshold)
    
    print(f"\nRESULTADOS DE SEPARACIÓN:")
    print(f"Total observaciones: {analysis['total_count']}")
    print(f"\nRÉGIMEN BAJO (≤{threshold}):")
    print(f"  Observaciones: {analysis['low_regime']['count']} ({analysis['low_regime']['proportion']:.1%})")
    print(f"  Media: {analysis['low_regime']['mean']:.2f}")
    print(f"  Std: {analysis['low_regime']['std']:.2f}")
    
    print(f"\nRÉGIMEN ALTO (>{threshold}):")
    print(f"  Observaciones: {analysis['high_regime']['count']} ({analysis['high_regime']['proportion']:.1%})")
    print(f"  Media: {analysis['high_regime']['mean']:.2f}")
    print(f"  Std: {analysis['high_regime']['std']:.2f}")
    
    # Visualización
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Histograma completo
    axes[0,0].hist(data, bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[0,0].axvline(threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {threshold}')
    axes[0,0].set_title(f'Distribución Completa - {product_name}')
    axes[0,0].set_xlabel('Ventas')
    axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].legend()
    
    # Régimen bajo con múltiples distribuciones
    if len(analysis['low_regime']['data']) > 0:
        low_data = analysis['low_regime']['data']
        n, bins, patches = axes[0,1].hist(low_data, bins=20, alpha=0.7, color='green', edgecolor='black', density=True)
        
        # 1. Poisson
        poisson_lambda = low_data.mean()
        x_discrete = np.arange(0, int(low_data.max()) + 1)
        y_poisson = stats.poisson.pmf(x_discrete, poisson_lambda)
        axes[0,1].plot(x_discrete, y_poisson, 'ro-', linewidth=2, markersize=3, label=f'Poisson(λ={poisson_lambda:.1f})')
        
        # 2. Binomial Negativa (usando método de momentos)
        try:
            mean_low = low_data.mean()
            var_low = low_data.var()
            if var_low > mean_low:  # Solo si hay sobredispersión
                # Parámetros por método de momentos: n, p
                p_est = mean_low / var_low
                n_est = mean_low * p_est / (1 - p_est)
                if 0 < p_est < 1 and n_est > 0:
                    y_nbinom = stats.nbinom.pmf(x_discrete, n_est, p_est)
                    axes[0,1].plot(x_discrete, y_nbinom, 'bs-', linewidth=2, markersize=3, 
                                 label=f'Binomial Neg(n={n_est:.1f}, p={p_est:.2f})')
        except:
            pass
        
        # 3. Uniforme Discreta (aproximación)
        min_val = int(low_data.min())
        max_val = int(low_data.max())
        uniform_prob = 1.0 / (max_val - min_val + 1)
        x_uniform = np.arange(min_val, max_val + 1)
        y_uniform = np.full(len(x_uniform), uniform_prob)
        axes[0,1].plot(x_uniform, y_uniform, 'm-', linewidth=3, alpha=0.8, 
                     label=f'Uniforme({min_val},{max_val})')
        
        # 4. Uniforme continua (para referencia visual)
        x_cont = np.linspace(low_data.min(), low_data.max(), 100)
        y_uniform_cont = stats.uniform.pdf(x_cont, low_data.min(), low_data.max() - low_data.min())
        axes[0,1].plot(x_cont, y_uniform_cont, 'c--', linewidth=2, alpha=0.7, 
                     label=f'Uniforme Continua')
        
        axes[0,1].set_title(f'Régimen Bajo (≤{threshold}) - Comparación Distribuciones')
        axes[0,1].set_xlabel('Ventas')
        axes[0,1].set_ylabel('Densidad')
        axes[0,1].legend(fontsize=8)
        axes[0,1].grid(True, alpha=0.3)
    
    # Régimen alto con Normal ajustada
    if len(analysis['high_regime']['data']) > 0:
        high_data = analysis['high_regime']['data']
        n, bins, patches = axes[1,0].hist(high_data, bins=20, alpha=0.7, color='orange', edgecolor='black', density=True)
        
        # Ajustar Normal
        normal_mu = high_data.mean()
        normal_sigma = high_data.std()
        x_normal = np.linspace(high_data.min(), high_data.max(), 100)
        y_normal = stats.norm.pdf(x_normal, normal_mu, normal_sigma)
        axes[1,0].plot(x_normal, y_normal, 'b-', linewidth=2, label=f'Normal(μ={normal_mu:.1f}, σ={normal_sigma:.1f})')
        
        axes[1,0].set_title(f'Régimen Alto (>{threshold}) vs Normal')
        axes[1,0].set_xlabel('Ventas')
        axes[1,0].set_ylabel('Densidad')
        axes[1,0].legend()
    
    # Comparación de distribuciones
    axes[1,1].hist(analysis['low_regime']['data'], bins=20, alpha=0.5, color='green', label=f'Bajo (≤{threshold})', density=True)
    axes[1,1].hist(analysis['high_regime']['data'], bins=20, alpha=0.5, color='orange', label=f'Alto (>{threshold})', density=True)
    axes[1,1].set_title('Comparación de Regímenes (Densidad)')
    axes[1,1].set_xlabel('Ventas')
    axes[1,1].set_ylabel('Densidad')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Test distribuciones
    if len(analysis['low_regime']['data']) > 10:
        print(f"\nTEST DE DISTRIBUCIONES - RÉGIMEN BAJO:")
        low_tests = test_distributions(analysis['low_regime']['data'], "low")
        for dist_name, result in low_tests.items():
            if isinstance(result, dict) and 'p_value' in result:
                print(f"  {dist_name.upper()}: KS p-value = {result['p_value']:.4f}")
                if dist_name == 'poisson':
                    print(f"    Lambda estimado: {result['param']:.2f}")
                elif dist_name == 'normal':
                    print(f"    Media: {result['params'][0]:.2f}, Std: {result['params'][1]:.2f}")
    
    if len(analysis['high_regime']['data']) > 10:
        print(f"\nTEST DE DISTRIBUCIONES - RÉGIMEN ALTO:")
        high_tests = test_distributions(analysis['high_regime']['data'], "high")
        for dist_name, result in high_tests.items():
            if isinstance(result, dict) and 'p_value' in result:
                print(f"  {dist_name.upper()}: KS p-value = {result['p_value']:.4f}")
                if dist_name == 'normal':
                    print(f"    Media: {result['params'][0]:.2f}, Std: {result['params'][1]:.2f}")
    
    return analysis

# ===== 4. ANÁLISIS PARA MÚLTIPLES THRESHOLDS =====
def find_optimal_threshold(data, threshold_range=range(50, 100, 5)):
    """Encuentra el threshold óptimo basado en criterios estadísticos"""
    
    results = []
    
    for threshold in threshold_range:
        analysis = analyze_threshold_split(data, threshold)
        
        # Criterios para evaluar la separación
        # 1. Balance en las proporciones (evitar que un grupo sea muy pequeño)
        balance_score = 1 - abs(0.5 - analysis['low_regime']['proportion'])
        
        # 2. Diferencia en medias (queremos que los grupos sean bien diferentes)
        if analysis['high_regime']['count'] > 0 and analysis['low_regime']['count'] > 0:
            mean_diff = abs(analysis['high_regime']['mean'] - analysis['low_regime']['mean'])
        else:
            mean_diff = 0
        
        # 3. Ratio de medias
        if analysis['low_regime']['mean'] > 0:
            mean_ratio = analysis['high_regime']['mean'] / analysis['low_regime']['mean']
        else:
            mean_ratio = 0
        
        results.append({
            'threshold': threshold,
            'low_prop': analysis['low_regime']['proportion'],
            'high_prop': analysis['high_regime']['proportion'],
            'low_mean': analysis['low_regime']['mean'],
            'high_mean': analysis['high_regime']['mean'],
            'mean_diff': mean_diff,
            'mean_ratio': mean_ratio,
            'balance_score': balance_score
        })
    
    results_df = pd.DataFrame(results)
    return results_df

# ===== EJECUCIÓN DEL ANÁLISIS =====

# Obtener lista de productos
products = train1['product'].unique()
countries = train1['country'].unique()

print("PRODUCTOS DISPONIBLES:")
for i, product in enumerate(products):
    print(f"{i+1}. {product}")

print("\nPAÍSES DISPONIBLES:")
for i, country in enumerate(countries):
    print(f"{i+1}. {country}")

# Análisis para el primer producto como ejemplo
if len(products) > 0:
    example_product = products[0]
    example_country = countries[0]
    
    print(f"\n🔍 ANÁLISIS DE EJEMPLO:")
    print(f"Producto: {example_product}")
    print(f"País: {example_country}")
    
    # Análisis con threshold 74
    analysis_74 = analyze_product_mixture(train1, example_product, example_country, threshold=74)
    
    # Buscar threshold óptimo
    product_data = train1[(train1['product'] == example_product) & 
                         (train1['country'] == example_country)]['num_sold']
    
    print(f"\n🎯 BÚSQUEDA DE THRESHOLD ÓPTIMO:")
    threshold_analysis = find_optimal_threshold(product_data)
    
    # Mostrar los mejores thresholds
    threshold_analysis['score'] = (threshold_analysis['balance_score'] * 
                                  threshold_analysis['mean_diff'] / 100)
    
    best_thresholds = threshold_analysis.nlargest(3, 'score')
    print("\nTOP 3 THRESHOLDS:")
    for idx, row in best_thresholds.iterrows():
        print(f"  Threshold: {row['threshold']}")
        print(f"    Proporción bajo: {row['low_prop']:.1%}, alto: {row['high_prop']:.1%}")
        print(f"    Media bajo: {row['low_mean']:.1f}, alto: {row['high_mean']:.1f}")
        print(f"    Ratio medias: {row['mean_ratio']:.2f}")
        print()



In [ ]:
# se ve , regimen bajo =<74 66% es binomial negativa o 
#         regiment alto >74 34% es normal
# mejor dicho se puede adepatar de esa manera
# este caso es para Argentina con el producto "Using LLMs to Improve Your Coding"


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats

def analyze_regime_threshold(data, threshold_candidates=range(10, 200, 10)):
    """Encuentra el mejor threshold para separar regímenes (tu función original)."""
    results = []
    for threshold in threshold_candidates:
        low_regime = data[data <= threshold]
        high_regime = data[data > threshold]
        
        if len(low_regime) < 10 or len(high_regime) < 10:
            continue
            
        low_prop = len(low_regime) / len(data)
        
        # Criterios para evaluar la separación
        balance_score = 1 - abs(0.5 - low_prop)
        mean_separation = abs(high_regime.mean() - low_regime.mean())
        cv_low = low_regime.std() / low_regime.mean() if low_regime.mean() > 0 else 99
        
        # Score combinado
        combined_score = (balance_score * 0.3 + 
                          (mean_separation / 100) * 0.5 + 
                          (1 / (1 + abs(cv_low - 1))) * 0.2)
        
        results.append({
            'threshold': threshold,
            'low_prop': low_prop,
            'high_prop': 1 - low_prop,
            'low_mean': low_regime.mean(),
            'high_mean': high_regime.mean(),
            'low_cv': cv_low,
            'mean_separation': mean_separation,
            'combined_score': combined_score
        })
    
    if not results:
        return None
    
    results_df = pd.DataFrame(results)
    best_threshold = results_df.loc[results_df['combined_score'].idxmax(), 'threshold']
    
    return best_threshold, results_df

def test_regime_distributions(low_data, high_data):
    """Testa qué distribuciones se ajustan mejor a cada régimen (tu función original)."""
    results = {'low_regime': {}, 'high_regime': {}}
    
    # Test régimen bajo
    if len(low_data) > 10:
        # Poisson
        try:
            poisson_lambda = low_data.mean()
            poisson_ks = stats.kstest(low_data, lambda x: stats.poisson.cdf(x, poisson_lambda))
            results['low_regime']['poisson'] = {'lambda': poisson_lambda, 'ks_pvalue': poisson_ks.pvalue}
        except: pass
        
        # Binomial Negativa
        try:
            mean_val, var_val = low_data.mean(), low_data.var()
            if var_val > mean_val:
                p_est = mean_val / var_val
                n_est = mean_val * p_est / (1 - p_est)
                if 0 < p_est < 1 and n_est > 0:
                    nbinom_ks = stats.kstest(low_data, lambda x: stats.nbinom.cdf(x, n_est, p_est))
                    results['low_regime']['nbinom'] = {'n': n_est, 'p': p_est, 'ks_pvalue': nbinom_ks.pvalue}
        except: pass
    
    # Test régimen alto
    if len(high_data) > 10:
        # Normal
        try:
            normal_mu, normal_sigma = stats.norm.fit(high_data)
            normal_ks = stats.kstest(high_data, lambda x: stats.norm.cdf(x, normal_mu, normal_sigma))
            results['high_regime']['normal'] = {'mu': normal_mu, 'sigma': normal_sigma, 'ks_pvalue': normal_ks.pvalue}
        except: pass
        
    return results

def plot_regime_distributions(data, threshold, low_regime, high_regime):
    """
    NUEVA FUNCIÓN: Genera un gráfico 2x2 para visualizar los regímenes y
    comparar las distribuciones.
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'Análisis de Distribución con Threshold = {threshold}', fontsize=16)

    # 1. Histograma completo
    axes[0, 0].hist(data, bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[0, 0].axvline(threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {threshold}')
    axes[0, 0].set_title('Distribución Completa de Ventas')
    axes[0, 0].set_xlabel('Ventas')
    axes[0, 0].set_ylabel('Frecuencia')
    axes[0, 0].legend()

    # 2. Régimen bajo vs. Distribuciones de Conteo
    if not low_regime.empty:
        axes[0, 1].hist(low_regime, bins=20, alpha=0.7, color='green', edgecolor='black', density=True, label='Datos Observados')
        
        # Curva Poisson
        poisson_lambda = low_regime.mean()
        x_poisson = np.arange(0, int(low_regime.max()) + 1)
        y_poisson = stats.poisson.pmf(x_poisson, poisson_lambda)
        axes[0, 1].plot(x_poisson, y_poisson, 'ro-', linewidth=2, markersize=4, label=f'Poisson (λ={poisson_lambda:.2f})')
        
        # Curva Binomial Negativa
        mean, var = low_regime.mean(), low_regime.var()
        if var > mean:
            p = mean / var
            n = mean * p / (1 - p)
            if 0 < p < 1 and n > 0:
                y_nbinom = stats.nbinom.pmf(x_poisson, n, p)
                axes[0, 1].plot(x_poisson, y_nbinom, 'bs-', linewidth=2, markersize=4, label=f'B. Negativa (n={n:.1f}, p={p:.2f})')

        axes[0, 1].set_title(f'Régimen Bajo (≤{threshold})')
        axes[0, 1].set_xlabel('Ventas')
        axes[0, 1].set_ylabel('Densidad')
        axes[0, 1].legend()

    # 3. Régimen alto vs. Distribución Normal
    if not high_regime.empty:
        axes[1, 0].hist(high_regime, bins=30, alpha=0.7, color='orange', edgecolor='black', density=True, label='Datos Observados')
        
        # Curva Normal
        mu, sigma = stats.norm.fit(high_regime)
        x_normal = np.linspace(high_regime.min(), high_regime.max(), 100)
        y_normal = stats.norm.pdf(x_normal, mu, sigma)
        axes[1, 0].plot(x_normal, y_normal, 'b-', linewidth=2, label=f'Normal (μ={mu:.1f}, σ={sigma:.1f})')
        
        axes[1, 0].set_title(f'Régimen Alto (>{threshold})')
        axes[1, 0].set_xlabel('Ventas')
        axes[1, 0].set_ylabel('Densidad')
        axes[1, 0].legend()

    # 4. Comparación de densidades
    if not low_regime.empty:
        low_regime.plot(kind='kde', ax=axes[1, 1], color='green', label=f'Bajo (≤{threshold})')
    if not high_regime.empty:
        high_regime.plot(kind='kde', ax=axes[1, 1], color='orange', label=f'Alto (>{threshold})')
    axes[1, 1].set_title('Comparación de Densidades (KDE)')
    axes[1, 1].set_xlabel('Ventas')
    axes[1, 1].legend()
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


# ===== ANÁLISIS COMPLETO POR PAÍS-PRODUCTO (MODIFICADO) =====

# Cargar datos (necesitas tener tu dataframe 'train1' cargado)
# train1 = pd.read_csv('tu_archivo.csv') # Ejemplo de carga

countries = train1['country'].unique()
products = train1['product'].unique()

regime_analysis_results = {}
optimal_thresholds = {}

print("=== ANÁLISIS DE REGÍMENES Y VISUALIZACIÓN ===\n")

for country in countries:
    print(f"{'='*60}\nPAÍS: {country.upper()}\n{'='*60}")
    
    regime_analysis_results[country] = {}
    optimal_thresholds[country] = {}
    
    for product in products:
        print(f"\n--- Producto: {product} ---")
        
        data = train1[(train1['country'] == country) & (train1['product'] == product)]['num_sold']
        
        if len(data) < 20: # Aumentamos el mínimo para un análisis robusto
            print(f"No hay datos suficientes ({len(data)} obs.)")
            continue
        
        print(f"Total observaciones: {len(data)}, Rango: {data.min()}-{data.max()}, Media: {data.mean():.1f}")
        
        # Encontrar threshold óptimo
        threshold_result = analyze_regime_threshold(data)
        
        if threshold_result is None:
            print("No se pudo encontrar un threshold adecuado.")
            continue
        
        best_threshold, _ = threshold_result
        optimal_thresholds[country][product] = best_threshold
        
        print(f"\n🎯 THRESHOLD ÓPTIMO ENCONTRADO: {best_threshold}")
        
        # Separar en regímenes
        low_regime = data[data <= best_threshold]
        high_regime = data[data > best_threshold]
        
        # Test de distribuciones
        dist_results = test_regime_distributions(low_regime, high_regime)
        regime_analysis_results[country][product] = {'threshold': best_threshold, 'distributions': dist_results}

        # Mostrar mejor ajuste y generar gráfico
        print("\n📊 MEJOR AJUSTE DE DISTRIBUCIONES (basado en p-valor de KS test):")
        # (Aquí puedes agregar la lógica para imprimir el mejor ajuste si lo deseas)

        # ¡NUEVO! Generar y mostrar el gráfico
        plot_regime_distributions(data, best_threshold, low_regime, high_regime)



In [ ]:
# el caso de argentina primero si ajusta bien ese producto , ahora hay algunos que si y otros no , el threshold va cambiando bastante
# en algunos casos me lleva a pensar en otras distribuciones
# para el primer modelo va sin regimen


# abajo primero resumen de todos los efectos que se vieron y todos los cambios a efectuar para un modelo simple 



In [ ]:
#1. ESTACIONALIDAD SEMANAL
# Factores de crecimiento semanal identificados
"""
weekend_boost = {
    0: 1.00,  # Lunes (base)
    1: 1.00,  # Martes  
    2: 1.00,  # Miércoles
    3: 1.00,  # Jueves
    4: 1.06,  # Viernes (+5-7% vs jueves)
    5: 1.09,  # Sábado (+3-5% vs viernes) 
    6: 1.17   # Domingo (+6-8% vs sábado)
}
train1['weekend_factor'] = train1['DayOfWeek'].map(weekend_boost)

# Features categóricas adicionales
train1['is_weekend'] = (train1['DayOfWeek'] >= 5).astype(int)
train1['is_friday'] = (train1['DayOfWeek'] == 4).astype(int)
"""
#2. EFECTOS FIJOS POR PAÍS (Multiplicadores constantes)
"""
# Multiplicadores identificados (Argentina = base)
country_multipliers = {
    'Argentina': 1.0,
    'Canada': 4.1, 
    'Estonia': 2.1,
    'Japan': 3.6,
    'Spain': 2.6
}
train1['country_multiplier'] = train1['country'].map(country_multipliers)

# One-hot encoding para efectos fijos
train1 = pd.get_dummies(train1, columns=['country'], prefix='country')
"""
#3. EFECTOS FIJOS POR TIENDA (Multiplicadores identificados)
"""
# Multiplicadores por tienda (Kaggle Learn = base)
store_multipliers = {
    'Kaggle Learn': 1.0,      # Base
    'Kaggle Store': 1.54,     # 1.54x base
    'Kagglazon': 5.67         # 5.67x base  
}
train1['store_multiplier'] = train1['store'].map(store_multipliers)

# One-hot encoding para tiendas
train1 = pd.get_dummies(train1, columns=['store'], prefix='store')
"""
#4. EFECTOS FIJOS POR PRODUCTO (Ranking consistente)
"""
# Ranking de productos (mismo orden en todos los países)
product_ranking = {
    'Using LLMs to Improve Your Coding': 1,
    'Using LLMs to Train More LLMs': 2, 
    'Using LLMs to Win More Kaggle Competitions': 3,
    'Using LLMs to Write Better': 4,
    'Using LLMs to Win Friends and Influence People': 5
}
train1['product_rank'] = train1['product'].map(product_ranking)

# Valores base por producto (usando Argentina + Kaggle Learn como referencia)
product_base_values = {
    'Using LLMs to Improve Your Coding': 82.3,
    'Using LLMs to Train More LLMs': 81.2,
    'Using LLMs to Win More Kaggle Competitions': 68.8, 
    'Using LLMs to Write Better': 62.6,
    'Using LLMs to Win Friends and Influence People': 12.4
}
train1['product_base_value'] = train1['product'].map(product_base_values)

# One-hot encoding
train1 = pd.get_dummies(train1, columns=['product'], prefix='product')
"""
#5. FEATURES TEMPORALES
"""
# Tendencias temporales
train1['year_trend'] = train1['Year'] - 2017  # 0,1,2,3,4
train1['days_since_start'] = (train1['date'] - train1['date'].min()).dt.days

# Seasonality dentro del año
train1['month_sin'] = np.sin(2 * np.pi * train1['Month'] / 12)
train1['month_cos'] = np.cos(2 * np.pi * train1['Month'] / 12)
train1['day_of_year'] = train1['date'].dt.dayofyear
train1['quarter_sin'] = np.sin(2 * np.pi * train1['Quarter'] / 4)
train1['quarter_cos'] = np.cos(2 * np.pi * train1['Quarter'] / 4)
"""
#6. MIXTURE MODEL FEATURES (Régimen identificado) no utilizado

#7. FEATURES DE INTERACCIÓN (Actualizadas con tienda)
"""
# Interacciones clave identificadas
train1['country_store'] = train1['country_multiplier'] * train1['store_multiplier']
train1['country_weekend'] = train1['country_multiplier'] * train1['weekend_factor']
train1['store_weekend'] = train1['store_multiplier'] * train1['weekend_factor']
train1['product_weekend'] = train1['product_base_value'] * train1['weekend_factor']
train1['product_store'] = train1['product_base_value'] * train1['store_multiplier']

# Interacción triple principal
train1['country_store_product'] = (train1['country_multiplier'] * 
                                  train1['store_multiplier'] * 
                                  train1['product_base_value'])

# Interacción año-producto (por si hay trends específicos)
for year in [2017, 2018, 2019, 2020, 2021]:
    train1[f'product_rank_year_{year}'] = (train1['product_rank'] * 
                                          (train1['Year'] == year).astype(int))
"""
#8. LAG FEATURES (Series temporales)
"""
# Ordenar por entidad y fecha
train1 = train1.sort_values(['country', 'store', 'product', 'date'])

# Lags por entidad (país-tienda-producto)
for lag in [1, 7, 30]:
    train1[f'num_sold_lag_{lag}'] = (train1.groupby(['country', 'store', 'product'])
                                    ['num_sold'].shift(lag))

# Rolling statistics
for window in [7, 30]:
    train1[f'num_sold_rolling_mean_{window}'] = (train1.groupby(['country', 'store', 'product'])
                                                ['num_sold'].rolling(window).mean()
                                                .reset_index(0, drop=True))
    
    train1[f'num_sold_rolling_std_{window}'] = (train1.groupby(['country', 'store', 'product'])
                                               ['num_sold'].rolling(window).std()
                                               .reset_index(0, drop=True))
"""
#9. FEATURES TARGET TRANSFORMADAS
"""
# Para el régimen normal (alta demanda)
train1['log_num_sold'] = np.log1p(train1['num_sold'])
train1['sqrt_num_sold'] = np.sqrt(train1['num_sold'])

# Para análisis de residuos
train1['residual_from_base'] = train1['num_sold'] - train1['expected_base']
"""
#10. FEATURES FINALES PARA MODELO
"""
# Lista de features para usar en el modelo
base_features = [
    # Efectos fijos multiplicativos
    'country_multiplier', 'store_multiplier', 'product_base_value', 'product_rank',
    
    # Estacionalidad 
    'weekend_factor', 'is_weekend', 'is_friday',
    'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos',
    
    # Temporales
    'year_trend', 'days_since_start',
    
    # Interacciones clave
    'country_store', 'country_weekend', 'store_weekend', 
    'product_weekend', 'product_store', 'country_store_product',
    
    # Base esperada y desviaciones
    'expected_base', 'deviation_from_expected',
    
    # Lags (si disponibles)
    'num_sold_lag_1', 'num_sold_lag_7', 'num_sold_rolling_mean_7'
]

# Features categóricas (one-hot encoded)
categorical_features = [col for col in train1.columns 
                       if col.startswith(('country_', 'product_', 'store_'))]

# Features finales
model_features = base_features + categorical_features
"""
#Como quedaria
#Expected_Sales = Argentina_Base × Country_Mult × Store_Mult × Product_Base × Weekend_Factor


In [ ]:
#  ESTACIONALIDAD SEMANAL
# Factores de crecimiento semanal identificados
weekend_boost = {
    0: 1.00,  # Lunes (base)
    1: 1.00,  # Martes  
    2: 1.00,  # Miércoles
    3: 1.00,  # Jueves
    4: 1.06,  # Viernes (+5-7% vs jueves)
    5: 1.09,  # Sábado (+3-5% vs viernes) 
    6: 1.17   # Domingo (+6-8% vs sábado)
}
train1['weekend_factor'] = train1['DayOfWeek'].map(weekend_boost)
test1['weekend_factor'] = test1['DayOfWeek'].map(weekend_boost)
# Features categóricas adicionales
train1['is_weekend'] = (train1['DayOfWeek'] >= 5).astype(int)
train1['is_friday'] = (train1['DayOfWeek'] == 4).astype(int)
test1['is_weekend'] = (test1['DayOfWeek'] >= 5).astype(int)
test1['is_friday'] = (test1['DayOfWeek'] == 4).astype(int)



In [ ]:
# EFECTOS FIJOS POR PAÍS
# Multiplicadores identificados (Argentina = base)
country_multipliers = {
    'Argentina': 1.0,
    'Canada': 4.1, 
    'Estonia': 2.1,
    'Japan': 3.6,
    'Spain': 2.6
}
train1['country_multiplier'] = train1['country'].map(country_multipliers)
test1['country_multiplier'] = test1['country'].map(country_multipliers)
# One-hot encoding para efectos fijos
#train1 = pd.get_dummies(train1, columns=['country'], prefix='country')

In [ ]:
#  EFECTOS FIJOS POR TIENDA
# Multiplicadores por tienda (Kaggle Learn = base)
store_multipliers = {
    'Kaggle Learn': 1.0,      # Base
    'Kaggle Store': 1.54,     # 1.54x base
    'Kagglazon': 5.67         # 5.67x base  
}
train1['store_multiplier'] = train1['store'].map(store_multipliers)
test1['store_multiplier'] = test1['store'].map(store_multipliers)
# One-hot encoding para tiendas
#train1 = pd.get_dummies(train1, columns=['store'], prefix='store')

In [ ]:
#  EFECTOS FIJOS POR PRODUCTO
# Ranking de productos (mismo orden en todos los países)
product_ranking = {
    'Using LLMs to Improve Your Coding': 1,
    'Using LLMs to Train More LLMs': 2, 
    'Using LLMs to Win More Kaggle Competitions': 3,
    'Using LLMs to Write Better': 4,
    'Using LLMs to Win Friends and Influence People': 5
}
train1['product_rank'] = train1['product'].map(product_ranking)
test1['product_rank'] = test1['product'].map(product_ranking)
# Valores base por producto (usando Argentina + Kaggle Learn como referencia)
product_base_values = {
    'Using LLMs to Improve Your Coding': 82.3,
    'Using LLMs to Train More LLMs': 81.2,
    'Using LLMs to Win More Kaggle Competitions': 68.8, 
    'Using LLMs to Write Better': 62.6,
    'Using LLMs to Win Friends and Influence People': 12.4
}
train1['product_base_value'] = train1['product'].map(product_base_values)
train1['product_base_value'] = train1['product'].map(product_base_values)
test1['product_base_value'] = test1['product'].map(product_base_values)
test1['product_base_value'] = test1['product'].map(product_base_values)
# One-hot encoding
#train1 = pd.get_dummies(train1, columns=['product'], prefix='product')

In [ ]:
# FEATURES TEMPORALES
# Tendencias temporales
train1['year_trend'] = train1['Year'] - 2017  # 0,1,2,3,4
train1['days_since_start'] = (train1['date'] - train1['date'].min()).dt.days
test1['year_trend'] = test1['Year'] - 2017
train_min_date = pd.to_datetime('2017-01-01')
test1['days_since_start'] = (test1['date'] - train_min_date).dt.days

# Seasonality dentro del año
train1['month_sin'] = np.sin(2 * np.pi * train1['Month'] / 12)
train1['month_cos'] = np.cos(2 * np.pi * train1['Month'] / 12)
train1['day_of_year'] = train1['date'].dt.dayofyear
train1['quarter_sin'] = np.sin(2 * np.pi * train1['Quarter'] / 4)
train1['quarter_cos'] = np.cos(2 * np.pi * train1['Quarter'] / 4)

test1['month_sin'] = np.sin(2 * np.pi * test1['Month'] / 12)
test1['month_cos'] = np.cos(2 * np.pi * test1['Month'] / 12)
test1['day_of_year'] = test1['date'].dt.dayofyear
test1['quarter_sin'] = np.sin(2 * np.pi * test1['Quarter'] / 4)
test1['quarter_cos'] = np.cos(2 * np.pi * test1['Quarter'] / 4)

In [ ]:
#FEATURES DE INTERACCIÓN
# Interacciones clave identificadas
train1['country_store'] = train1['country_multiplier'] * train1['store_multiplier']
train1['country_weekend'] = train1['country_multiplier'] * train1['weekend_factor']
train1['store_weekend'] = train1['store_multiplier'] * train1['weekend_factor']
train1['product_weekend'] = train1['product_base_value'] * train1['weekend_factor']
train1['product_store'] = train1['product_base_value'] * train1['store_multiplier']

test1['country_store'] = test1['country_multiplier'] * test1['store_multiplier']
test1['country_weekend'] = test1['country_multiplier'] * test1['weekend_factor']
test1['store_weekend'] = test1['store_multiplier'] * test1['weekend_factor']
test1['product_weekend'] = test1['product_base_value'] * train1['weekend_factor']
test1['product_store'] = test1['product_base_value'] * test1['store_multiplier']




# Interacción triple principal
train1['country_store_product'] = (train1['country_multiplier'] * 
                                  train1['store_multiplier'] * 
                                  train1['product_base_value'])

test1['country_store_product'] = (test1['country_multiplier'] * 
                                  test1['store_multiplier'] * 
                                  test1['product_base_value'])


# Interacción año-producto (por si hay trends específicos)
for year in [2017, 2018, 2019, 2020, 2021]:
    train1[f'product_rank_year_{year}'] = (train1['product_rank'] * 
                                          (train1['Year'] == year).astype(int))

for year in [2017, 2018, 2019, 2020, 2021, 2022]:
        test1[f'product_rank_year_{year}'] = (test1['product_rank'] * 
                                                      (test1['Year'] == year).astype(int))


In [ ]:
# LAG FEATURES
# Ordenar por entidad y fecha
train1 = train1.sort_values(['country', 'store', 'product', 'date'])

# Lags por entidad (país-tienda-producto)
for lag in [1, 7, 30]:
    train1[f'num_sold_lag_{lag}'] = (train1.groupby(['country', 'store', 'product'])
                                    ['num_sold'].shift(lag))

# Rolling statistics
for window in [7, 30]:
    train1[f'num_sold_rolling_mean_{window}'] = (
        train1.groupby(['country', 'store', 'product'])['num_sold']
        .transform(lambda x: x.rolling(window).mean())
    )

    train1[f'num_sold_rolling_std_{window}'] = (
        train1.groupby(['country', 'store', 'product'])['num_sold']
        .transform(lambda x: x.rolling(window).std())
    )

test1 = test1.sort_values(['country', 'store', 'product', 'date'])

# Lags por entidad (país-tienda-producto)
for lag in [1, 7, 30]:
    test1[f'num_sold_lag_{lag}'] = (test1.groupby(['country', 'store', 'product'])
                                    ['num_sold'].shift(lag))

# Rolling statistics
for window in [7, 30]:
    test1[f'num_sold_rolling_mean_{window}'] = (
        test1.groupby(['country', 'store', 'product'])['num_sold']
        .transform(lambda x: x.rolling(window).mean())
    )

    test1[f'num_sold_rolling_std_{window}'] = (
        test1.groupby(['country', 'store', 'product'])['num_sold']
        .transform(lambda x: x.rolling(window).std())
    )


In [ ]:
# FEATURES TARGET TRANSFORMADAS
# Para el régimen normal (alta demanda)
train1['log_num_sold'] = np.log1p(train1['num_sold']) 
#train1['sqrt_num_sold'] = np.sqrt(train1['num_sold']) no lo uso x ahora

#one hot al final
#train1 = pd.get_dummies(train1, columns=['country'], prefix='country')
#train1 = pd.get_dummies(train1, columns=['store'], prefix='store')
#train1 = pd.get_dummies(train1, columns=['product'], prefix='product')

#test1 = pd.get_dummies(test1, columns=['country'], prefix='country')
#test1 = pd.get_dummies(test1, columns=['store'], prefix='store')
#test1 = pd.get_dummies(test1, columns=['product'], prefix='product')





In [ ]:
# FEATURES FINALES PARA MODELO
# Lista de features para usar en el modelo
base_features = [
    # Efectos fijos multiplicativos
    'country_multiplier', 'store_multiplier', 'product_base_value', 'product_rank',
    
    # Estacionalidad 
    'weekend_factor', 'is_weekend', 'is_friday',
    'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos',
    
    # Temporales
    'year_trend', 'days_since_start',
    
    # Interacciones clave
    'country_store', 'country_weekend', 'store_weekend', 
    'product_weekend', 'product_store', 'country_store_product',
    
    # Base esperada y desviaciones
    'expected_base', 'deviation_from_expected',
    
    # Lags (si disponibles)
    'num_sold_lag_1', 'num_sold_lag_7', 'num_sold_rolling_mean_7'
]

# Features categóricas (one-hot encoded)
categorical_features = [col for col in train1.columns 
                       if col.startswith(('country_', 'product_', 'store_'))]

# Features finales
model_features = base_features + categorical_features




In [ ]:
train1.head()

In [ ]:
test1.head()

In [ ]:
train1.info()

In [ ]:
test1.info()

In [ ]:
#eliminar los NaN/ reemplzar por 0
# no saque la columna ID que no aporta poder predictivo

features_con_nans = [col for col in train1.columns if 'lag' in col or 'rolling' in col]
train1[features_con_nans] = train1[features_con_nans].fillna(0)

In [ ]:


model_features = [
    'country_multiplier', 'store_multiplier', 'product_base_value', 'product_rank',
    'weekend_factor', 'is_weekend', 'is_friday',
    'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos',
    'year_trend', 'days_since_start',
    'country_store', 'country_weekend', 'store_weekend', 
    'product_weekend', 'product_store', 'country_store_product',
    'num_sold_lag_1', 'num_sold_lag_7', 'num_sold_lag_30',
    'num_sold_rolling_mean_7', 'num_sold_rolling_std_7',
    'num_sold_rolling_mean_30', 'num_sold_rolling_std_30'
]

X = train1[model_features]
y = train1['log_num_sold']

X_test = test1[model_features]

In [ ]:
#  fecha de corte voy a usar 3 meses solamente de validacion
fecha_corte = '2021-09-30'
indices_train = train1[train1['date'] < fecha_corte].index
indices_val = train1[train1['date'] >= fecha_corte].index

# DATA LEAKEAGE , despues termine viendo que las variables de lag y rolling producieron data LEAKEAGE
# asi que lo que probe en el test de validacion esta mal ,
# Separar X e y
X_train, X_val = X.loc[indices_train], X.loc[indices_val]
y_train, y_val = y.loc[indices_train], y.loc[indices_val]

print(f"Datos de entrenamiento: {len(X_train)} filas")
print(f"Datos de validación: {len(X_val)} filas")
"""
# Valores prediciendo los ultimos 3 meses del 2021

[LightGBM] [Info] Total Bins 2271
[LightGBM] [Info] Number of data points in the train set: 129975, number of used features: 26
[LightGBM] [Info] Start training from score 4.534541
Error del modelo (RMSE) en el conjunto de validación: 21.39
Error del modelo (SMAPE): 5.41%
"""

In [ ]:
X.info()

In [ ]:
X_test.info()

In [ ]:
lgbm_reg = lgb.LGBMRegressor(random_state=1722, n_jobs=-1)
lgbm_reg.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import mean_squared_error

# Hacer predicciones sobre el conjunto de validación
log_predicciones = lgbm_reg.predict(X_val)

# Convertir las predicciones de vuelta a la escala original
# Si tu target 'y' era log_num_sold, usá np.expm1()
predicciones_finales = np.expm1(log_predicciones)

# Obtener los valores reales también en la escala original
valores_reales = np.expm1(y_val)


rmse = np.sqrt(mean_squared_error(valores_reales, predicciones_finales))
print(f"\nError del modelo (RMSE) en el conjunto de validación: {rmse:.2f}")


plt.figure(figsize=(15, 6))
plt.plot(train1.loc[indices_val, 'date'], valores_reales, label='Valores Reales', alpha=0.7)
plt.plot(train1.loc[indices_val, 'date'], predicciones_finales, label='Predicciones del Modelo', linestyle='--')
plt.title('Comparación de Valores Reales vs. Predicciones')
plt.legend()
plt.show()

In [ ]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0  
    return np.mean(diff) * 100
# Calcular SMAPE
smape_error = smape(valores_reales, predicciones_finales)
print(f"Error del modelo (SMAPE): {smape_error:.2f}%")


In [ ]:
#Modelo para enviar a kaggle
# voy a tener que ir prediciendo de a 1 mes por la forma en que arme mis variables

lgbm_1 = lgb.LGBMRegressor(random_state=1722, n_jobs=-1)
lgbm_1.fit(X, y)



In [ ]:
# Dividir X_test por meses usando days_since_start
monthly_tests = {}
inicio_2022 = 1795  # tu valor mínimo
dias_acumulados = 0
dias_por_mes = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

for month in range(1, 13):
   inicio_mes = inicio_2022 + dias_acumulados
   fin_mes = inicio_2022 + dias_acumulados + dias_por_mes[month-1] - 1
   
   mask = (X_test['days_since_start'] >= inicio_mes) & (X_test['days_since_start'] <= fin_mes)
   monthly_tests[f'month_{month}'] = X_test[mask]
   
   dias_acumulados += dias_por_mes[month-1]

# Inicializar historial con datos reales hasta diciembre 2021
historial = train1.copy()
predicciones_finales = []

# Loop iterativo
for month in range(1, 13):
    print(f"=== MES {month} ===")
    
    # Predecir mes actual
    X_mes_actual = monthly_tests[f'month_{month}']
    print(f"Filas a predecir: {len(X_mes_actual)}")
    
    pred_mes = lgbm_1.predict(X_mes_actual)
    print(f"Predicciones generadas: {len(pred_mes)}")
    print(f"Ejemplo predicciones: {pred_mes[:5]}")  # primeras 5
    
    # Agregar a lista final
    predicciones_finales.extend(pred_mes)
    
    # Agregar predicciones al historial para próximos lags
    # (actualizar historial con pred_mes) - POR IMPLEMENTAR
    
    # Recalcular lags para próximo mes  
    # (calcular nuevos lags usando historial actualizado) - POR IMPLEMENTAR

print(f"\n=== RESUMEN FINAL ===")  
print(f"Total predicciones: {len(predicciones_finales)}")
print(f"Primeras 10 predicciones: {predicciones_finales[:10]}")


In [ ]:
# 1. Aplicar exponencial a las predicciones
predicciones_finales_exp = [np.exp(pred) for pred in predicciones_finales]

# 2. Crear DataFrame con las predicciones
pred_df = pd.DataFrame({
    'num_sold': predicciones_finales_exp
})

# 3. Agregar las columnas originales desde test1
pred_df_completo = pd.concat([
    test1[['date', 'country', 'store', 'product']].reset_index(drop=True),
    pred_df
], axis=1)

# 4. Convertir fechas al mismo formato antes del merge
test_merge = test[['id', 'date', 'country', 'store', 'product']].copy()
test_merge['date'] = pd.to_datetime(test_merge['date'])
pred_df_completo['date'] = pd.to_datetime(pred_df_completo['date'])

# 5. Hacer merge
submission = test_merge.merge(
    pred_df_completo,
    on=['date', 'country', 'store', 'product'],
    how='left'
)

# 6. Submission final
submission_final = submission[['id', 'num_sold']]
print(f"Submission shape: {submission_final.shape}")
print(submission_final.head())

# 7. Guardar para Kaggle
submission_final.to_csv('submission.csv', index=False)

In [ ]:
print(f"Shape: {submission_final.shape}")
print(f"Columnas: {submission_final.columns.tolist()}")
print(f"Tipos: {submission_final.dtypes}")
print(f"Valores nulos: {submission_final.isnull().sum()}")
print(submission_final.head())

In [ ]:
#podria hacer ensemble/DNN y hacer que prediga 75 variables entonces es analizar cada pais/tienda/producto y ya sea hacer una variable aleatoria de c/u
# osea ejemplo ,para ARG/TIENDA1/Producto 1 , y esto es una variable conocida o combinacion o hago que la red aprenda como esta distribuido en los 
# cuartiles y asi la forme , pero la pag hoy no esta andando bien y no deja ver como quedo en relacion al leaderboard(fijo)
# tmb en este mismo notebook quedaron muchas cosas por probar , con las variables one-hot y mas